## Deliverable 2. Create a Customer Travel Destinations Map.

In [81]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import sys
sys.path.append('../')
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [82]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Akdepe,TM,42.0551,59.3788,75.29,28,0,10.07,clear sky
1,1,Mataura,NZ,-46.1927,168.8643,43.48,95,80,2.82,broken clouds
2,2,Tashla,RU,51.7716,52.7427,74.03,51,9,4.38,clear sky
3,3,Touros,BR,-5.1989,-35.4608,79.70,74,5,19.42,clear sky
4,4,Nyurba,RU,63.2842,118.3319,63.18,60,100,4.99,overcast clouds


In [51]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [52]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Akdepe,TM,42.0551,59.3788,75.29,28,0,10.07,clear sky
3,3,Touros,BR,-5.1989,-35.4608,79.70,74,5,19.42,clear sky
7,7,Marathon,US,44.9000,-89.7668,75.24,47,75,11.50,broken clouds
10,10,Atuona,PF,-9.8000,-139.0333,77.11,69,31,11.18,scattered clouds
12,12,Kapaa,US,22.0752,-159.3190,84.18,77,90,3.00,light rain
16,16,Avarua,CK,-21.2078,-159.7750,75.25,94,75,11.50,moderate rain
20,20,Lerici,IT,44.0759,9.9112,75.65,73,34,16.64,scattered clouds
27,27,Yellowknife,CA,62.4560,-114.3525,87.33,28,20,3.44,few clouds
32,32,Kirakira,SB,-10.4544,161.9205,78.39,75,88,8.48,overcast clouds
34,34,Axim,GH,4.8699,-2.2405,75.88,89,21,5.48,few clouds


In [53]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                245
City                   245
Country                245
Lat                    245
Lng                    245
Max Temp               245
Humidity               245
Cloudiness             245
Wind Speed             245
Current Description    245
dtype: int64

In [54]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Akdepe,TM,42.0551,59.3788,75.29,28,0,10.07,clear sky
3,3,Touros,BR,-5.1989,-35.4608,79.70,74,5,19.42,clear sky
7,7,Marathon,US,44.9000,-89.7668,75.24,47,75,11.50,broken clouds
10,10,Atuona,PF,-9.8000,-139.0333,77.11,69,31,11.18,scattered clouds
12,12,Kapaa,US,22.0752,-159.3190,84.18,77,90,3.00,light rain


In [55]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)
short_hotel = hotel_df.head(5)

In [56]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [72]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Akdepe,TM,75.29,clear sky,42.0551,59.3788,Мой дом
3,Touros,BR,79.70,clear sky,-5.1989,-35.4608,INN NEW HORIZON
7,Marathon,US,75.24,broken clouds,44.9000,-89.7668,
10,Atuona,PF,77.11,scattered clouds,-9.8000,-139.0333,Villa Enata
12,Kapaa,US,84.18,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [61]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
#this doesn't seem to work, so used next line

In [77]:
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Akdepe,TM,75.29,clear sky,42.0551,59.3788,Мой дом
3,Touros,BR,79.70,clear sky,-5.1989,-35.4608,INN NEW HORIZON
10,Atuona,PF,77.11,scattered clouds,-9.8000,-139.0333,Villa Enata
12,Kapaa,US,84.18,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
16,Avarua,CK,75.25,moderate rain,-21.2078,-159.7750,Paradise Inn


In [83]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [79]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [80]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))